In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import albumentations as A
import albumentations.pytorch
from tqdm.notebook import tqdm

import cv2
import re
import time

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.parallel import DataParallelModel, DataParallelCriterion

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print ('Available devices ', torch.cuda.device_count())

# print ('Current cuda device ', torch.cuda.current_device())
# print(torch.cuda.get_device_name(device))

# GPU 할당 변경하기
GPU_NUM = 4 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print(device)
print ('Current cuda device ', torch.cuda.current_device()) # check
device_ids = [4,0,2,3]

cuda:4
Current cuda device  4


In [3]:
# os.environ["CUDA_VISIBLE_DEVICES"] = '0, 2, 3, 4'
# print ('Current cuda device ', torch.cuda.current_device()) # check

In [4]:
# device = torch.device('cuda')
# device = torch.device('cpu')

In [5]:
# %time
# PATH_TO_WEIGHTS = '../coco_resnet_50_map_0_335_state_dict.pt'
# retinanet = model.resnet50(num_classes=5,)
# # retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS))

In [6]:
%time
PATH_TO_WEIGHTS = '../coco_resnet_50_map_0_335_state_dict.pt'
pretrained_retinanet = model.resnet50(num_classes=80, device=device)
pretrained_retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS))


retinanet = model.resnet50(num_classes=5, device=device)
for param, state in zip(pretrained_retinanet.parameters(), pretrained_retinanet.state_dict()) :
    #print(state)
    if 'classificationModel' not in state :
        retinanet.state_dict()[state] = param
    else :
        print(state)
    
for param, state in zip(pretrained_retinanet.fpn.parameters(), pretrained_retinanet.fpn.state_dict()) :
    #print(state)
    retinanet.fpn.state_dict()[state] = param

for param, state in zip(pretrained_retinanet.regressionModel.parameters(), pretrained_retinanet.regressionModel.state_dict()) :
    #print(state)
    retinanet.regressionModel.state_dict()[state] = param  

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 29.3 µs


In [7]:
# https://pypi.org/project/torch-encoding/
# torch-encoding should be installed
# pip install torch-encoding
# import torch.encoding as encoding
# retinanet = encoding.nn.DataParallelModel(retinanet, device_ids = [4,0,2,3]).to(device)

In [8]:

# retinanet.to(device)
retinanet = torch.nn.DataParallel(retinanet, device_ids = [4,0,2,3], output_device=4).to(device)
# retinanet = DataParallelModel(retinanet, device_ids = device_ids)
retinanet.to(device)
# retinanet.cuda()

# retinanet.module.freeze_bn()

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [9]:
train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

In [10]:
batch_size = 32
train_ds = PapsDataset(train_info, transforms)

train_data_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

In [11]:
criterion = FocalLoss(device)
# criterion = DataParallelCriterion(criterion, device_ids = device_ids) 
criterion = criterion.to(device)
# optimizer = optim.Adam(retinanet.parameters(), lr=1e-4)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
# retinanet.train()
retinanet.training = True
# retinanet.module.freeze_bn()    
# retinanet.freeze_bn()

In [12]:
# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(retinanet.parameters(), lr = 1e-7)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=1, eta_max=0.0005,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts


In [13]:
scheduler.state_dict()
# # scheduler._last_lr
# optimizer.param_groups[0]["lr"]

{'T_0': 20,
 'T_mult': 1,
 'base_eta_max': 0.0005,
 'eta_max': 0.0005,
 'T_up': 5,
 'T_i': 20,
 'gamma': 0.5,
 'cycle': 0,
 'T_cur': 0,
 'base_lrs': [1e-07],
 'last_epoch': 0,
 '_step_count': 0,
 'verbose': False}

In [ ]:
#for i, data in enumerate(tqdm(train_data_loader)) :
EPOCH_NUM = 60
loss_per_epoch = 2
for epoch in range(EPOCH_NUM) :
    epoch_loss = []
    total_loss = 0
    tk0 = tqdm(train_data_loader, total=len(train_data_loader), leave=False)
    EPOCH_LEARING_RATE = optimizer.param_groups[0]["lr"]
    print("*****{}th epoch, learning rate {}".format(epoch, EPOCH_LEARING_RATE))
    
    for step, data in enumerate(tk0) :
        images, _, paths, targets = data
#         print(targets)
        batch_size = len(images)

    #     images = list(image.to(device) for image in images)
        c, h, w = images[0].shape
        images = torch.cat(images).view(-1, c, h, w).to(device)
#         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        targets = [ t.to(device) for t in targets]

#         classification_loss, regression_loss = retinanet([images, targets])
        outputs = retinanet([images, targets])
        classification, regression, anchors, annotations = (outputs)
        classification_loss, regression_loss = criterion(classification, regression, anchors, annotations)

#         output = retinanet(images)
#         features, regression, classification = output
#         classification_loss, regression_loss = criterion(classification, regression, modified_anchors, targets)    
        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()
        loss = classification_loss + regression_loss 
        total_loss += loss.item()
        
        epoch_loss.append((loss.item()))
        tk0.set_postfix(lr=optimizer.param_groups[0]["lr"], batch_loss=loss.item(), cls_loss=classification_loss.item(), 
                        reg_loss=regression_loss.item(), avg_loss=total_loss/(step+1))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)
        optimizer.step()   

    print('{}th epochs loss is {}'.format(epoch, np.mean(epoch_loss)))
    if loss_per_epoch > np.mean(epoch_loss):
        print('best model is saved')
        torch.save(retinanet.state_dict(), 'best_model.pt')
        loss_per_epoch = np.mean(epoch_loss)
#     scheduler.step(np.mean(epoch_loss))
    scheduler.step()


  0%|          | 0/468 [00:00<?, ?it/s]

*****0th epoch, learning rate 1e-07


In [ ]:
torch.save(retinanet.state_dict(), '../trained_models/model.pt')

In [ ]:
retinanet.load_state_dict(torch.load('../trained_models/model.pt'))

In [ ]:
# retinanet.eval()

In [ ]:
test_info = np.load('../data/test.npy', allow_pickle=True, encoding='latin1').item()
# train_info

In [ ]:
test_ds = PapsDataset(test_info,val_transforms)

test_data_loader = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
retinanet.eval()
# retinanet.training = True
tk1 = tqdm(test_data_loader, total=len(test_data_loader),leave=False)
for step, data in enumerate(tk1) :
    with torch.no_grad():
        images, _, paths, targets = data
        batch_size = len(images)
        c, h, w = images[0].shape
        images = torch.cat(images).view(-1, c, h, w).to(device)
        print(images.shape)
        targets = [ t.to(device) for t in targets]   
        
        scores, labels, boxes = retinanet(images)
        print(scores)
        print(labels)
        print(boxes)
        adfsfd
    

In [ ]:
retinanet.eval()
retinanet.training